In [1]:
import pandas as pd
import numpy as np 

In [2]:
# cgm_sensor_data = pd.read_csv('CGMData.csv',low_memory=False) - this is the normal read csv command
# but according to the requirements given, the output of the CGM sensor consists of three columns:  Data time stamp (Columns B and C combined), the 5 minute filtered CGM reading in mg/dL, (Column AE) and the ISIG value which is the raw sensor output every 5 mins.
# so now instead of reading the whole cgm sensor data csv file, we need to read only these four columns first. 
cgm_sensor_data = pd.read_csv('CGMData.csv', low_memory=False, usecols=['Date', 'Time', 'Sensor Glucose (mg/dL)', 'ISIG Value'])

In [3]:
cgm_sensor_data

,Date,Time,Sensor Glucose (mg/dL),ISIG Value
0,2/12/2018,13:22:27,118.0,15.16
1,2/12/2018,13:17:27,122.0,16.12
2,2/12/2018,13:12:27,NaN,NaN
3,2/12/2018,13:07:27,NaN,NaN
4,2/12/2018,13:02:27,NaN,NaN
...,...,...,...,...
55338,7/25/2017,12:28:54,311.0,44.91
55339,7/25/2017,12:23:54,311.0,44.26
55340,7/25/2017,12:18:54,309.0,44.51
55341,7/25/2017,12:13:54,310.0,45.16


In [4]:
# since it is specified that we need to use the date time stamp column as one - which is by combining the date and time columns
# so we need to now create a new column in the creatde cgm_sensor_data dataframe to store the combined value of the date time.
# but since in the created data frame the values stored in the date and time columns are in a string format. 
# so we need to convert them into time objects so that the further calculations like time comparing or with the insulin date time later on. 
# and that is why I am using the Pandas to_datetime() method which helps to convert string Date time into Python Date time object.
cgm_sensor_data['date_time_stamp'] = pd.to_datetime(cgm_sensor_data['Date'] + ' ' + cgm_sensor_data['Time'])

In [5]:
cgm_sensor_data

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
0,2/12/2018,13:22:27,118.0,15.16,2018-02-12 13:22:27
1,2/12/2018,13:17:27,122.0,16.12,2018-02-12 13:17:27
2,2/12/2018,13:12:27,NaN,NaN,2018-02-12 13:12:27
3,2/12/2018,13:07:27,NaN,NaN,2018-02-12 13:07:27
4,2/12/2018,13:02:27,NaN,NaN,2018-02-12 13:02:27
...,...,...,...,...,...
55338,7/25/2017,12:28:54,311.0,44.91,2017-07-25 12:28:54
55339,7/25/2017,12:23:54,311.0,44.26,2017-07-25 12:23:54
55340,7/25/2017,12:18:54,309.0,44.51,2017-07-25 12:18:54
55341,7/25/2017,12:13:54,310.0,45.16,2017-07-25 12:13:54


In [6]:
# it is mentioned in the requirements that you have to tackle the “missing data problem”
# In the data files, those are represented as NaN. 
# You need to devise a strategy to tackle the missing data problem. 
# Popular strategies include deletion of the entire day of data, or interpolation. 

In [7]:
# so now to check the number of NaN values in each column of the cgm_sensor_data data frame. 
cgm_sensor_data.isnull().sum(axis = 0)

Date                         0
Time                         0
Sensor Glucose (mg/dL)    4168
ISIG Value                4168
date_time_stamp              0
dtype: int64

In [8]:
# from the above output it is evident that there are no null values in the date and time columns
# and because of the same reason there are also no null values in the manually creates date_time_stamp column too. 
# but there are equal or same number of null values in the Sensor Glucose (mg/dL) and the ISIG value columns. 

# therefore, now as suggested in the requirements, the best way to deal with the null values is either deletion or interpolation
# I would like to follow the deletion method to deal with the null values occuring in the cgm_sensor_data data frame. 

In [9]:
# since now I decided to follow with the deletion method to deal with the null values
# and it is clearly mentioned that - we need to delete the data for the entire day. 

# so from the above output of 4168 null values in total there might cases where some days might have multiple instances of NaN
# whereas there might be some days there is only one instance of NaN. 
# therefore, in any case we need to delete the data for the entire day. 
# so for that I need to figure out all the days or dates that have atleast one instance of missing data.

In [10]:
cgm_sensor_data[cgm_sensor_data['Sensor Glucose (mg/dL)'].isna()]

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
2,2/12/2018,13:12:27,NaN,NaN,2018-02-12 13:12:27
3,2/12/2018,13:07:27,NaN,NaN,2018-02-12 13:07:27
4,2/12/2018,13:02:27,NaN,NaN,2018-02-12 13:02:27
12,2/12/2018,12:22:27,NaN,NaN,2018-02-12 12:22:27
19,2/12/2018,11:47:27,NaN,NaN,2018-02-12 11:47:27
...,...,...,...,...,...
54267,7/29/2017,5:44:15,NaN,NaN,2017-07-29 05:44:15
54268,7/29/2017,5:39:15,NaN,NaN,2017-07-29 05:39:15
54414,7/28/2017,17:29:12,NaN,NaN,2017-07-28 17:29:12
54874,7/27/2017,3:09:02,NaN,NaN,2017-07-27 03:09:02


In [11]:
# so the above code command returns the total 4168 rows which have the NaN values.
# but now from this we need to extract all the dates that have the missing values. 
# if one day has multiple instances of missing data, even then we need to return the date only once. 
# so basically we need to extract all the unique dates which have missing values. 
dates_to_be_removed_from_the_cgm_sensor_data = cgm_sensor_data[cgm_sensor_data['Sensor Glucose (mg/dL)'].isna()]['Date'].unique()
dates_to_be_removed_from_the_cgm_sensor_data

array(['2/12/2018', '2/11/2018', '2/10/2018', '2/9/2018', '2/8/2018',
       '2/7/2018', '2/6/2018', '2/5/2018', '2/4/2018', '2/3/2018',
       '2/2/2018', '2/1/2018', '1/31/2018', '1/30/2018', '1/29/2018',
       '1/28/2018', '1/27/2018', '1/26/2018', '1/25/2018', '1/24/2018',
       '1/23/2018', '1/22/2018', '1/20/2018', '1/17/2018', '1/16/2018',
       '1/15/2018', '1/14/2018', '1/13/2018', '1/12/2018', '1/11/2018',
       '1/1/2018', '12/31/2017', '12/30/2017', '12/29/2017', '12/28/2017',
       '12/27/2017', '12/26/2017', '12/25/2017', '12/24/2017',
       '12/22/2017', '12/21/2017', '12/20/2017', '12/19/2017',
       '12/17/2017', '12/16/2017', '12/15/2017', '12/14/2017',
       '12/13/2017', '12/10/2017', '12/9/2017', '12/8/2017', '12/7/2017',
       '12/6/2017', '12/5/2017', '12/4/2017', '12/2/2017', '11/30/2017',
       '11/29/2017', '11/28/2017', '11/27/2017', '11/26/2017',
       '11/25/2017', '11/23/2017', '11/22/2017', '11/21/2017',
       '11/20/2017', '11/19/2017', '11/1

In [12]:
len(dates_to_be_removed_from_the_cgm_sensor_data)

158

In [13]:
# so now from the above output you can see that there are a total of 158 days which have atleast one instance of missing values. 
# so now we need to remove the data with respect to all these dates even the for the 5 min instances for which data was available.

In [14]:
# this code block is just to check how many rows might be eliminated if delete the data for all the dates present in the dates_to_be_removed_from_the_cgm_sensor_data array

# cgm_sensor_data.loc[cgm_sensor_data['Date'] == '2/12/2018']
cgm_sensor_data.loc[cgm_sensor_data['Date'].isin(dates_to_be_removed_from_the_cgm_sensor_data)]

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
0,2/12/2018,13:22:27,118.0,15.16,2018-02-12 13:22:27
1,2/12/2018,13:17:27,122.0,16.12,2018-02-12 13:17:27
2,2/12/2018,13:12:27,NaN,NaN,2018-02-12 13:12:27
3,2/12/2018,13:07:27,NaN,NaN,2018-02-12 13:07:27
4,2/12/2018,13:02:27,NaN,NaN,2018-02-12 13:02:27
...,...,...,...,...,...
55338,7/25/2017,12:28:54,311.0,44.91,2017-07-25 12:28:54
55339,7/25/2017,12:23:54,311.0,44.26,2017-07-25 12:23:54
55340,7/25/2017,12:18:54,309.0,44.51,2017-07-25 12:18:54
55341,7/25/2017,12:13:54,310.0,45.16,2017-07-25 12:13:54


In [15]:
# so again from the above output, we can see that a total of 44,958 rows will be deleted from the available data.

In [16]:
# so now to delete all the rows corresponding to all the dates present in the dates_to_be_removed_from_the_cgm_sensor_data list.
# the first step for this deletion process would be set up an index for the data frame so that we could handle operations 
# like deletion, lookup etc in the optimal way manner and this helps in this case because we are reading some heavy data. 
cgm_sensor_data = cgm_sensor_data.set_index('Date')
cgm_sensor_data
# I have a set an index on the date column for the cgm_sensor_data data frame because I have the date_time_stamp column for my further calculations.

,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
Date,,,,
2/12/2018,13:22:27,118.0,15.16,2018-02-12 13:22:27
2/12/2018,13:17:27,122.0,16.12,2018-02-12 13:17:27
2/12/2018,13:12:27,NaN,NaN,2018-02-12 13:12:27
2/12/2018,13:07:27,NaN,NaN,2018-02-12 13:07:27
2/12/2018,13:02:27,NaN,NaN,2018-02-12 13:02:27
...,...,...,...,...
7/25/2017,12:28:54,311.0,44.91,2017-07-25 12:28:54
7/25/2017,12:23:54,311.0,44.26,2017-07-25 12:23:54
7/25/2017,12:18:54,309.0,44.51,2017-07-25 12:18:54


In [17]:
# now since the index for the cgm_sensor_data data frame is set on the date column
# we can now just use this index to drop all those rows whose index value is matching any one of the values in the dates_to_be_removed_from_the_cgm_sensor_data list
cgm_sensor_data = cgm_sensor_data.drop(index=dates_to_be_removed_from_the_cgm_sensor_data)
cgm_sensor_data

,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
Date,,,,
1/21/2018,23:59:18,91.0,18.20,2018-01-21 23:59:18
1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18
1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
...,...,...,...,...
7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57


In [18]:
# now reset the index back so that we can have the date column back again in the data frame for any further calculations
cgm_sensor_data = cgm_sensor_data.reset_index()
cgm_sensor_data

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
0,1/21/2018,23:59:18,91.0,18.20,2018-01-21 23:59:18
1,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18
2,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
3,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
4,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
...,...,...,...,...,...
10380,7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
10381,7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
10382,7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57
10383,7/26/2017,0:08:57,64.0,14.26,2017-07-26 00:08:57


In [19]:
# making a copy of the cgm_sensor_data data frame for further calculations 
# this is because I would have the original processed and cleaned CGM Sensor Data Frame intact without any modifications
# which would help to use this in the further calculation if required and until then use the cloned data frame to work on. 
cgm_sensor_data_copy_test = cgm_sensor_data.copy()
cgm_sensor_data_copy_test

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
0,1/21/2018,23:59:18,91.0,18.20,2018-01-21 23:59:18
1,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18
2,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
3,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
4,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
...,...,...,...,...,...
10380,7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
10381,7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
10382,7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57
10383,7/26/2017,0:08:57,64.0,14.26,2017-07-26 00:08:57


In [20]:
# now again we need to index this data frame for optimal lookups 
# since we want to have the date, time, and the date_time_stamps columns intact in the data frame
# we can create the index for this cloned data frame by using the date_time_stamp column of the original data frame. 
# therefore, this would make the column date_time_stamp as the index for the cloned data frame 
# but we would still have the date_time_stamp column intact in the cloned data frame as we have used the date_time_stamp column from the original data frame.
cgm_sensor_data_copy_test = cgm_sensor_data_copy_test.set_index(pd.DatetimeIndex(cgm_sensor_data['date_time_stamp']))
cgm_sensor_data_copy_test

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
date_time_stamp,,,,,
2018-01-21 23:59:18,1/21/2018,23:59:18,91.0,18.20,2018-01-21 23:59:18
2018-01-21 23:54:18,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18
2018-01-21 23:49:18,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
2018-01-21 23:44:18,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
2018-01-21 23:39:18,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
...,...,...,...,...,...
2017-07-26 00:23:57,7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
2017-07-26 00:18:57,7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
2017-07-26 00:13:57,7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57


In [21]:
insulin_pump_data = pd.read_csv('InsulinData.csv', low_memory=False)

In [22]:
insulin_pump_data

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Linked BG Meter ID,Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),...,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Preset Bolus,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,Sensor Exception,Preset Temp Basal Name
0,0,2/12/2018,13:20:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN,NaN,NaN
1,1,2/12/2018,13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2/12/2018,13:18:48,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2/12/2018,13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2/12/2018,13:12:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41430,22135,7/24/2017,19:00:01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41431,22136,7/24/2017,18:59:44,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41432,22137,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41433,22138,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# since it is specified that we need to use the date time stamp column as one - which is by combining the date and time columns
# so we need to now create a new column in the created insulin_pump_data dataframe to store the combined value of the date time.
# but since in the created data frame the values stored in the date and time columns are in a string format. 
# so we need to convert them into time objects so that the further calculations like time comparing etc. 
# and that is why I am using the Pandas to_datetime() method which helps to convert string Date time into Python Date time object.
insulin_pump_data['date_time_stamp'] = pd.to_datetime(insulin_pump_data['Date'] + ' ' + insulin_pump_data['Time'])

In [24]:
insulin_pump_data

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Linked BG Meter ID,Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),...,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Preset Bolus,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,Sensor Exception,Preset Temp Basal Name,date_time_stamp
0,0,2/12/2018,13:20:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN,NaN,NaN,2018-02-12 13:20:53
1,1,2/12/2018,13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:20:48
2,2,2/12/2018,13:18:48,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
3,3,2/12/2018,13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
4,4,2/12/2018,13:12:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:12:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41430,22135,7/24/2017,19:00:01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 19:00:01
41431,22136,7/24/2017,18:59:44,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41432,22137,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41433,22138,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44


In [25]:
# so now we have read the CGM Sensor Data and the Insulin Pump Data into data frames
# we also have a processed and cleaned CGM Sensor Data frame
# so now we have to find the 1st instance of the AUTO MODE ACTIVE PLGM OFF in the Alarm column in the Insulin Pump Data data frame

# but again in the requirements it is mentioned that The data is in reverse order of time. 
# This means that the first row is the end of the data collection whereas the last row is the beginning of the data collection.
# so now for us to find the 1st instance of the AUTO MODE ACTIVE PLGM OFF in the Alarm column in the Insulin Pump Data data frame
# we have to sort the entire Insulin Pump Data data frame according to the date_time_stamp column in the ascending order

insulin_pump_data = insulin_pump_data.sort_values(by = 'date_time_stamp', ascending = True)
insulin_pump_data

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Linked BG Meter ID,Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),...,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Preset Bolus,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,Sensor Exception,Preset Temp Basal Name,date_time_stamp
41434,22139,7/24/2017,18:59:42,7/24/2017 18:59,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:42
41431,22136,7/24/2017,18:59:44,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41433,22138,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41432,22137,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41430,22135,7/24/2017,19:00:01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 19:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,4,2/12/2018,13:12:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:12:33
3,3,2/12/2018,13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
2,2,2/12/2018,13:18:48,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
1,1,2/12/2018,13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:20:48


In [26]:
# now from this sorted Insulin Pump Data data frame 
# we need to now figure out the 1st instance of the AUTO MODE ACTIVE PLGM OFF in the Alarm column in the Insulin Pump Data data frame

# so for this let us extract out all the rows from the Insulin Pump Data data frame where the Alarm column has the value = AUTO MODE ACTIVE PLGM OFF

insulin_pump_data_alarm_column_data = insulin_pump_data.loc[insulin_pump_data['Alarm'] == 'AUTO MODE ACTIVE PLGM OFF']
insulin_pump_data_alarm_column_data

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Linked BG Meter ID,Basal Rate (U/h),Temp Basal Amount,Temp Basal Type,Temp Basal Duration (h:mm:ss),...,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Preset Bolus,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,Sensor Exception,Preset Temp Basal Name,date_time_stamp
40131,20836,8/9/2017,8:07:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-09 08:07:13
20449,1154,11/4/2017,17:36:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-04 17:36:46


In [27]:
# so now to get the 1st instance of the AUTO MODE ACTIVE PLGM OFF in the Alarm column in the Insulin Pump Data data frame
# we just need to extract the date_time_stamp of the 1st row from the above created insulin_pump_data_alarm_column_data dataframe
# this is because the insulin_pump_data dataframe was sorted in ascending order 
# and therefore, by deafult the insulin_pump_data_alarm_column_data dataframe will also be sorted in ascending order 
# and therfore, the date time stamp of the first row is also the 1st instance of the AUTO MODE ACTIVE PLGM OFF in the Alarm column in the Insulin Pump Data data frame

start_of_auto_mode = insulin_pump_data_alarm_column_data.iloc[0]['date_time_stamp']
start_of_auto_mode

Timestamp('2017-08-09 08:07:13')

In [28]:
# now again it is mentioned in the requirements that Once you determine the start of Auto Mode from InsulinData.csv, 
# you have to figure out the timestamp in CGMData.csv where Auto mode starts. 
# This can be done simply by searching for the time stamp that is nearest to (and later than) the Auto mode start time stamp obtained from InsulinData.csv.

# so now for this we now need to sort the CGM sensor data in ascending order. 

In [29]:
# we are now again creating a copy of the cgm_sensor_dataframe because of two reasons
# 1. we ofcourse do want to modify the processed and cleaned original CGM sensor dataframe in any manner
# 2. the previously cloned data frame - cgm_sensor_data_copy_test - cannot be used to for sorting
#    a. this is becasue the cgm_sensor_data_copy_test dataframe has an index with respect to the date_time_stamp column.
#    b. and if in this case if execute cgm_sensor_data_copy_test.sort_values(by = 'date_time_stamp', ascending = True)
#    c. we would get an error - "ambiguous" because there is an index with date_time_stamp and there is also a date_time_stamp column. 
# that is why we are creating a new clone so that we could sort the CGM Sensor data in the ascending order with respect to the date_time_stamp column
cgm_sensor_data_clone = cgm_sensor_data.copy()
cgm_sensor_data_clone

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
0,1/21/2018,23:59:18,91.0,18.20,2018-01-21 23:59:18
1,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18
2,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
3,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
4,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
...,...,...,...,...,...
10380,7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
10381,7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
10382,7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57
10383,7/26/2017,0:08:57,64.0,14.26,2017-07-26 00:08:57


In [30]:
sorted_cgm_sensor_data = cgm_sensor_data_clone.sort_values(by = 'date_time_stamp', ascending = True)
sorted_cgm_sensor_data

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
10384,7/26/2017,0:03:57,65.0,14.45,2017-07-26 00:03:57
10383,7/26/2017,0:08:57,64.0,14.26,2017-07-26 00:08:57
10382,7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57
10381,7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
10380,7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
...,...,...,...,...,...
4,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
3,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
2,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
1,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18


In [31]:
# now we need to split the whole sorted_cgm_sensor_data dataframe into two
# values collected in the auto mode and the values collected manually. 
# since the data collection starts manually and only shifts to auto mode only when the AUTO MODE ACTIVE PLGM OFF is activated

# and we have found the 1st instance when this AUTO MODE ACTIVE PLGM OFF is activated in the Insulin pump data dataframe. 
# and using that timestamp we need to segregate the sorted_cgm_sensor_data into parts. 

# so all the datetime stamps which are lesser than the datetime stamp of the 1st instance when this AUTO MODE ACTIVE PLGM OFF - will be placed in to the manually collected data set
# and all the datetime stamps which are greater than or equal to the datetime stamp of the 1st instance when this AUTO MODE ACTIVE PLGM OFF - will be placed in to the auto mode collected data set

auto_mode_data_collected_dataframe = sorted_cgm_sensor_data.loc[sorted_cgm_sensor_data['date_time_stamp'] >= start_of_auto_mode]

manual_mode_data_collected_dataframe = sorted_cgm_sensor_data.loc[sorted_cgm_sensor_data['date_time_stamp'] < start_of_auto_mode]

In [32]:
auto_mode_data_collected_dataframe

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
9134,8/9/2017,8:10:05,173.0,32.03,2017-08-09 08:10:05
9133,8/9/2017,8:15:05,177.0,31.38,2017-08-09 08:15:05
9132,8/9/2017,8:20:05,176.0,31.77,2017-08-09 08:20:05
9131,8/9/2017,8:25:05,175.0,31.44,2017-08-09 08:25:05
9130,8/9/2017,8:30:05,177.0,31.87,2017-08-09 08:30:05
...,...,...,...,...,...
4,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
3,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
2,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
1,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18


In [33]:
manual_mode_data_collected_dataframe

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
10384,7/26/2017,0:03:57,65.0,14.45,2017-07-26 00:03:57
10383,7/26/2017,0:08:57,64.0,14.26,2017-07-26 00:08:57
10382,7/26/2017,0:13:57,64.0,14.47,2017-07-26 00:13:57
10381,7/26/2017,0:18:57,67.0,15.06,2017-07-26 00:18:57
10380,7/26/2017,0:23:57,72.0,15.55,2017-07-26 00:23:57
...,...,...,...,...,...
9139,8/9/2017,7:45:05,160.0,31.81,2017-08-09 07:45:05
9138,8/9/2017,7:50:05,157.0,31.56,2017-08-09 07:50:05
9137,8/9/2017,7:55:05,158.0,32.12,2017-08-09 07:55:05
9136,8/9/2017,8:00:05,159.0,32.27,2017-08-09 08:00:05


Metrics extracted for the data collected in the Auto Mode

In [34]:
auto_mode_data_collected_dataframe_date_index = auto_mode_data_collected_dataframe.copy()
auto_mode_data_collected_dataframe_date_index

,Date,Time,Sensor Glucose (mg/dL),ISIG Value,date_time_stamp
9134,8/9/2017,8:10:05,173.0,32.03,2017-08-09 08:10:05
9133,8/9/2017,8:15:05,177.0,31.38,2017-08-09 08:15:05
9132,8/9/2017,8:20:05,176.0,31.77,2017-08-09 08:20:05
9131,8/9/2017,8:25:05,175.0,31.44,2017-08-09 08:25:05
9130,8/9/2017,8:30:05,177.0,31.87,2017-08-09 08:30:05
...,...,...,...,...,...
4,1/21/2018,23:39:18,101.0,20.02,2018-01-21 23:39:18
3,1/21/2018,23:44:18,98.0,19.49,2018-01-21 23:44:18
2,1/21/2018,23:49:18,95.0,19.04,2018-01-21 23:49:18
1,1/21/2018,23:54:18,93.0,18.59,2018-01-21 23:54:18


In [35]:
auto_mode_data_collected_dataframe_date_index = auto_mode_data_collected_dataframe_date_index.set_index('date_time_stamp')
auto_mode_data_collected_dataframe_date_index

,Date,Time,Sensor Glucose (mg/dL),ISIG Value
date_time_stamp,,,,
2017-08-09 08:10:05,8/9/2017,8:10:05,173.0,32.03
2017-08-09 08:15:05,8/9/2017,8:15:05,177.0,31.38
2017-08-09 08:20:05,8/9/2017,8:20:05,176.0,31.77
2017-08-09 08:25:05,8/9/2017,8:25:05,175.0,31.44
2017-08-09 08:30:05,8/9/2017,8:30:05,177.0,31.87
...,...,...,...,...
2018-01-21 23:39:18,1/21/2018,23:39:18,101.0,20.02
2018-01-21 23:44:18,1/21/2018,23:44:18,98.0,19.49
2018-01-21 23:49:18,1/21/2018,23:49:18,95.0,19.04


In [36]:
list1 = auto_mode_data_collected_dataframe_date_index.groupby('Date')['Sensor Glucose (mg/dL)'].count().where(lambda x:x>0.8*288).dropna().index.tolist()
list1

['1/18/2018',
 '1/19/2018',
 '1/21/2018',
 '10/11/2017',
 '10/20/2017',
 '10/26/2017',
 '10/27/2017',
 '11/11/2017',
 '11/12/2017',
 '11/24/2017',
 '11/9/2017',
 '12/1/2017',
 '12/11/2017',
 '12/12/2017',
 '12/18/2017',
 '12/23/2017',
 '12/3/2017',
 '8/12/2017',
 '8/19/2017',
 '8/24/2017',
 '8/30/2017',
 '8/31/2017',
 '9/10/2017',
 '9/14/2017',
 '9/15/2017',
 '9/16/2017',
 '9/17/2017',
 '9/21/2017',
 '9/23/2017',
 '9/3/2017',
 '9/4/2017']

In [37]:
auto_mode_data_collected_dataframe_date_index = auto_mode_data_collected_dataframe_date_index.loc[auto_mode_data_collected_dataframe_date_index['Date'].isin(list1)]
auto_mode_data_collected_dataframe_date_index

,Date,Time,Sensor Glucose (mg/dL),ISIG Value
date_time_stamp,,,,
2017-08-12 00:03:31,8/12/2017,0:03:31,148.0,30.18
2017-08-12 00:08:31,8/12/2017,0:08:31,147.0,29.78
2017-08-12 00:13:31,8/12/2017,0:13:31,143.0,29.23
2017-08-12 00:18:31,8/12/2017,0:18:31,137.0,28.68
2017-08-12 00:23:31,8/12/2017,0:23:31,131.0,28.11
...,...,...,...,...
2018-01-21 23:39:18,1/21/2018,23:39:18,101.0,20.02
2018-01-21 23:44:18,1/21/2018,23:44:18,98.0,19.49
2018-01-21 23:49:18,1/21/2018,23:49:18,95.0,19.04


Percentage time in hyperglycemia (CGM > 180 mg/dL) - wholeday, daytime, overnight
- For data collected in Auto Mode

In [38]:
percentage_time_in_hyperglycemia_during_wholeday_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] > 180]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100)
percentage_time_in_hyperglycemia_during_wholeday_when_data_collected_in_auto_mode

Date
1/18/2018     15.625000
1/19/2018     25.694444
1/21/2018     28.819444
10/11/2017    35.416667
10/20/2017    27.083333
10/26/2017    24.652778
10/27/2017    19.097222
11/11/2017     2.430556
11/12/2017     4.166667
11/24/2017    27.430556
11/9/2017     33.333333
12/1/2017     17.361111
12/11/2017    15.625000
12/12/2017    15.277778
12/18/2017    18.750000
12/23/2017    30.902778
12/3/2017      7.291667
8/12/2017     26.388889
8/19/2017     40.277778
8/24/2017     52.083333
8/30/2017     10.416667
8/31/2017     30.555556
9/10/2017      5.902778
9/14/2017     37.152778
9/15/2017     27.083333
9/16/2017     22.569444
9/17/2017      8.333333
9/21/2017      7.638889
9/23/2017     17.708333
9/3/2017      31.597222
9/4/2017      32.291667
Name: Sensor Glucose (mg/dL), dtype: float64

In [39]:
percentage_time_in_hyperglycemia_during_daytime_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] > 180]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
    )
percentage_time_in_hyperglycemia_during_daytime_when_data_collected_in_auto_mode

Date
1/18/2018     12.847222
1/19/2018     25.694444
1/21/2018     26.041667
10/11/2017    35.416667
10/20/2017    27.083333
10/26/2017    20.138889
10/27/2017    19.097222
11/11/2017     2.430556
11/12/2017     4.166667
11/24/2017    13.888889
11/9/2017     28.819444
12/1/2017     17.361111
12/11/2017    10.069444
12/12/2017    15.277778
12/18/2017    17.361111
12/23/2017    26.736111
12/3/2017      7.291667
8/12/2017     26.388889
8/19/2017     35.069444
8/24/2017     36.458333
8/30/2017      8.680556
8/31/2017     30.555556
9/10/2017      5.902778
9/14/2017     29.166667
9/15/2017     27.083333
9/16/2017     22.569444
9/17/2017      8.333333
9/21/2017      7.638889
9/23/2017     17.708333
9/3/2017      31.597222
9/4/2017      17.013889
Name: Sensor Glucose (mg/dL), dtype: float64

In [40]:
percentage_time_in_hyperglycemia_during_overnight_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] > 180]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_during_overnight_when_data_collected_in_auto_mode

Date
1/18/2018      2.777778
1/21/2018      2.777778
10/26/2017     4.513889
11/24/2017    13.541667
11/9/2017      4.513889
12/11/2017     5.555556
12/18/2017     1.388889
12/23/2017     4.166667
8/19/2017      5.208333
8/24/2017     15.625000
8/30/2017      1.736111
9/14/2017      7.986111
9/4/2017      15.277778
Name: Sensor Glucose (mg/dL), dtype: float64

percentage of time in hyperglycemia critical (CGM > 250 mg/dL) - wholeday, daytime, overnight
- For data collected in Auto Mode

In [41]:
percentage_time_in_hyperglycemia_critical_during_wholeday_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] > 250]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_critical_during_wholeday_when_data_collected_in_auto_mode

Date
1/18/2018      6.944444
1/19/2018      5.555556
1/21/2018      5.555556
10/11/2017    15.625000
10/26/2017     3.819444
10/27/2017     5.555556
11/9/2017     10.069444
12/11/2017     2.777778
12/18/2017     2.083333
12/23/2017    13.541667
8/12/2017     12.847222
8/19/2017     10.763889
8/24/2017      4.513889
8/31/2017      6.944444
9/15/2017      4.861111
9/16/2017     10.069444
9/17/2017      3.819444
9/23/2017      1.388889
9/3/2017      19.444444
9/4/2017      14.583333
Name: Sensor Glucose (mg/dL), dtype: float64

In [42]:
percentage_time_in_hyperglycemia_critical_during_daytime_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] > 250]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_critical_during_daytime_when_data_collected_in_auto_mode

Date
1/18/2018      6.944444
1/19/2018      5.555556
1/21/2018      5.555556
10/11/2017    15.625000
10/26/2017     1.388889
10/27/2017     5.555556
11/9/2017      9.027778
12/11/2017     2.777778
12/18/2017     2.083333
12/23/2017    13.541667
8/12/2017     12.847222
8/19/2017     10.763889
8/24/2017      4.513889
8/31/2017      6.944444
9/15/2017      4.861111
9/16/2017     10.069444
9/17/2017      3.819444
9/23/2017      1.388889
9/3/2017      19.444444
9/4/2017       3.819444
Name: Sensor Glucose (mg/dL), dtype: float64

In [43]:
percentage_time_in_hyperglycemia_critical_during_overnight_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] > 250]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_critical_during_overnight_when_data_collected_in_auto_mode

Date
10/26/2017     2.430556
11/9/2017      1.041667
9/4/2017      10.763889
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL) - wholeday, daytime, overnight
- For data collected in Auto Mode

In [44]:
percentage_time_in_range_during_wholeday_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] >= 70) & (auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] <= 180)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_during_wholeday_when_data_collected_in_auto_mode

Date
1/18/2018     79.861111
1/19/2018     74.305556
1/21/2018     65.277778
10/11/2017    64.583333
10/20/2017    69.791667
10/26/2017    70.833333
10/27/2017    77.777778
11/11/2017    94.444444
11/12/2017    90.625000
11/24/2017    72.569444
11/9/2017     59.722222
12/1/2017     82.638889
12/11/2017    77.777778
12/12/2017    79.513889
12/18/2017    78.472222
12/23/2017    63.541667
12/3/2017     83.680556
8/12/2017     73.611111
8/19/2017     53.472222
8/24/2017     47.916667
8/30/2017     85.416667
8/31/2017     64.236111
9/10/2017     91.319444
9/14/2017     62.847222
9/15/2017     71.875000
9/16/2017     77.430556
9/17/2017     90.625000
9/21/2017     86.805556
9/23/2017     78.125000
9/3/2017      54.166667
9/4/2017      53.472222
Name: Sensor Glucose (mg/dL), dtype: float64

In [45]:
percentage_time_in_range_during_daytime_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] >= 70) & (auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] <= 180)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_during_daytime_when_data_collected_in_auto_mode

Date
1/18/2018     59.375000
1/19/2018     49.305556
1/21/2018     43.055556
10/11/2017    39.583333
10/20/2017    44.791667
10/26/2017    54.861111
10/27/2017    52.777778
11/11/2017    69.444444
11/12/2017    65.625000
11/24/2017    61.111111
11/9/2017     39.236111
12/1/2017     57.638889
12/11/2017    59.722222
12/12/2017    54.513889
12/18/2017    57.638889
12/23/2017    42.708333
12/3/2017     58.680556
8/12/2017     48.611111
8/19/2017     33.680556
8/24/2017     38.541667
8/30/2017     64.930556
8/31/2017     39.236111
9/10/2017     66.319444
9/14/2017     45.833333
9/15/2017     46.875000
9/16/2017     52.430556
9/17/2017     65.625000
9/21/2017     61.805556
9/23/2017     53.125000
9/3/2017      29.861111
9/4/2017      43.750000
Name: Sensor Glucose (mg/dL), dtype: float64

In [46]:
percentage_time_in_range_during_overnight_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] >= 70) & (auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] <= 180)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_during_overnight_when_data_collected_in_auto_mode

Date
1/18/2018     20.486111
1/19/2018     25.000000
1/21/2018     22.222222
10/11/2017    25.000000
10/20/2017    25.000000
10/26/2017    15.972222
10/27/2017    25.000000
11/11/2017    25.000000
11/12/2017    25.000000
11/24/2017    11.458333
11/9/2017     20.486111
12/1/2017     25.000000
12/11/2017    18.055556
12/12/2017    25.000000
12/18/2017    20.833333
12/23/2017    20.833333
12/3/2017     25.000000
8/12/2017     25.000000
8/19/2017     19.791667
8/24/2017      9.375000
8/30/2017     20.486111
8/31/2017     25.000000
9/10/2017     25.000000
9/14/2017     17.013889
9/15/2017     25.000000
9/16/2017     25.000000
9/17/2017     25.000000
9/21/2017     25.000000
9/23/2017     25.000000
9/3/2017      24.305556
9/4/2017       9.722222
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL) - wholeday, daytime, overnight

- For data collected in Auto Mode

In [47]:
percentage_time_in_range_secondary_during_wholeday_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] >= 70) & (auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] <= 150)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_secondary_during_wholeday_when_data_collected_in_auto_mode

Date
1/18/2018     54.166667
1/19/2018     58.680556
1/21/2018     44.097222
10/11/2017    41.666667
10/20/2017    55.902778
10/26/2017    54.513889
10/27/2017    62.500000
11/11/2017    87.847222
11/12/2017    71.527778
11/24/2017    45.486111
11/9/2017     51.041667
12/1/2017     60.069444
12/11/2017    58.333333
12/12/2017    66.666667
12/18/2017    57.638889
12/23/2017    50.347222
12/3/2017     61.111111
8/12/2017     64.930556
8/19/2017     34.375000
8/24/2017     34.722222
8/30/2017     57.291667
8/31/2017     53.819444
9/10/2017     62.500000
9/14/2017     38.888889
9/15/2017     51.736111
9/16/2017     72.222222
9/17/2017     71.180556
9/21/2017     73.958333
9/23/2017     67.361111
9/3/2017      44.791667
9/4/2017      49.652778
Name: Sensor Glucose (mg/dL), dtype: float64

In [48]:
percentage_time_in_range_secondary_during_daytime_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] >= 70) & (auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] <= 150)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_secondary_during_daytime_when_data_collected_in_auto_mode

Date
1/18/2018     38.194444
1/19/2018     39.583333
1/21/2018     26.736111
10/11/2017    22.916667
10/20/2017    34.027778
10/26/2017    39.583333
10/27/2017    38.888889
11/11/2017    62.847222
11/12/2017    49.305556
11/24/2017    45.486111
11/9/2017     34.375000
12/1/2017     44.444444
12/11/2017    46.180556
12/12/2017    41.666667
12/18/2017    43.402778
12/23/2017    35.416667
12/3/2017     36.111111
8/12/2017     39.930556
8/19/2017     19.791667
8/24/2017     28.819444
8/30/2017     42.013889
8/31/2017     28.819444
9/10/2017     37.500000
9/14/2017     26.388889
9/15/2017     29.166667
9/16/2017     47.222222
9/17/2017     49.305556
9/21/2017     50.694444
9/23/2017     42.361111
9/3/2017      25.000000
9/4/2017      42.013889
Name: Sensor Glucose (mg/dL), dtype: float64

In [49]:
percentage_time_in_range_secondary_during_overnight_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] >= 70) & (auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] <= 150)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_secondary_during_overnight_when_data_collected_in_auto_mode

Date
1/18/2018     15.972222
1/19/2018     19.097222
1/21/2018     17.361111
10/11/2017    18.750000
10/20/2017    21.875000
10/26/2017    14.930556
10/27/2017    23.611111
11/11/2017    25.000000
11/12/2017    22.222222
11/9/2017     16.666667
12/1/2017     15.625000
12/11/2017    12.152778
12/12/2017    25.000000
12/18/2017    14.236111
12/23/2017    14.930556
12/3/2017     25.000000
8/12/2017     25.000000
8/19/2017     14.583333
8/24/2017      5.902778
8/30/2017     15.277778
8/31/2017     25.000000
9/10/2017     25.000000
9/14/2017     12.500000
9/15/2017     22.569444
9/16/2017     25.000000
9/17/2017     21.875000
9/21/2017     23.263889
9/23/2017     25.000000
9/3/2017      19.791667
9/4/2017       7.638889
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in hypoglycemia level 1 (CGM < 70 mg/dL) - wholeday, daytime, overnight

- For data collected in Auto Mode

In [50]:
percentage_time_in_hypoglycemia_level1_during_wholeday_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] < 70]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level1_during_wholeday_when_data_collected_in_auto_mode

Date
1/18/2018      4.513889
1/21/2018      5.902778
10/20/2017     3.125000
10/26/2017     4.513889
10/27/2017     3.125000
11/11/2017     3.125000
11/12/2017     5.208333
11/9/2017      6.944444
12/11/2017     6.597222
12/12/2017     5.208333
12/18/2017     2.777778
12/23/2017     5.555556
12/3/2017      9.027778
8/19/2017      6.250000
8/30/2017      4.166667
8/31/2017      5.208333
9/10/2017      2.777778
9/15/2017      1.041667
9/17/2017      1.041667
9/21/2017      5.555556
9/23/2017      4.166667
9/3/2017      14.236111
9/4/2017      14.236111
Name: Sensor Glucose (mg/dL), dtype: float64

In [51]:
percentage_time_in_hypoglycemia_level1_during_daytime_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] < 70]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level1_during_daytime_when_data_collected_in_auto_mode

Date
1/18/2018      2.777778
1/21/2018      5.902778
10/20/2017     3.125000
10/27/2017     3.125000
11/11/2017     3.125000
11/12/2017     5.208333
11/9/2017      6.944444
12/11/2017     5.208333
12/12/2017     5.208333
12/23/2017     5.555556
12/3/2017      9.027778
8/19/2017      6.250000
8/30/2017      1.388889
8/31/2017      5.208333
9/10/2017      2.777778
9/15/2017      1.041667
9/17/2017      1.041667
9/21/2017      5.555556
9/23/2017      4.166667
9/3/2017      13.541667
9/4/2017      14.236111
Name: Sensor Glucose (mg/dL), dtype: float64

In [52]:
percentage_time_in_hypoglycemia_level1_during_overnight_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] < 70]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level1_during_overnight_when_data_collected_in_auto_mode

Date
1/18/2018     1.736111
10/26/2017    4.513889
12/11/2017    1.388889
12/18/2017    2.777778
8/30/2017     2.777778
9/3/2017      0.694444
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in hypoglycemia level 2 (CGM < 54 mg/dL) - wholeday, daytime, overnight

- For data collected in Auto Mode

In [53]:
percentage_time_in_hypoglycemia_level2_during_wholeday_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] < 54]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level2_during_wholeday_when_data_collected_in_auto_mode

Date
1/21/2018     1.736111
10/20/2017    2.083333
11/11/2017    0.694444
11/12/2017    3.472222
11/9/2017     3.472222
12/11/2017    1.388889
12/12/2017    0.347222
12/3/2017     3.819444
8/19/2017     1.041667
8/31/2017     1.041667
9/21/2017     0.347222
9/3/2017      1.041667
9/4/2017      0.694444
Name: Sensor Glucose (mg/dL), dtype: float64

In [54]:
percentage_time_in_hypoglycemia_level2_during_daytime_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] < 54]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level2_during_daytime_when_data_collected_in_auto_mode

Date
1/21/2018     1.736111
10/20/2017    2.083333
11/11/2017    0.694444
11/12/2017    3.472222
11/9/2017     3.472222
12/11/2017    1.388889
12/12/2017    0.347222
12/3/2017     3.819444
8/19/2017     1.041667
8/31/2017     1.041667
9/21/2017     0.347222
9/3/2017      1.041667
9/4/2017      0.694444
Name: Sensor Glucose (mg/dL), dtype: float64

In [55]:
percentage_time_in_hypoglycemia_level2_during_overnight_when_data_collected_in_auto_mode = (
    auto_mode_data_collected_dataframe_date_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[auto_mode_data_collected_dataframe_date_index['Sensor Glucose (mg/dL)'] < 54]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level2_during_overnight_when_data_collected_in_auto_mode

Series([], Name: Sensor Glucose (mg/dL), dtype: float64)

Metrics extracted for the data collected in Manual Mode

In [56]:
manual_mode_data_collected_dataframe_index = manual_mode_data_collected_dataframe.copy()
manual_mode_data_collected_dataframe_index = manual_mode_data_collected_dataframe.set_index('date_time_stamp')

In [57]:
manual_mode_data_collected_dataframe_index

,Date,Time,Sensor Glucose (mg/dL),ISIG Value
date_time_stamp,,,,
2017-07-26 00:03:57,7/26/2017,0:03:57,65.0,14.45
2017-07-26 00:08:57,7/26/2017,0:08:57,64.0,14.26
2017-07-26 00:13:57,7/26/2017,0:13:57,64.0,14.47
2017-07-26 00:18:57,7/26/2017,0:18:57,67.0,15.06
2017-07-26 00:23:57,7/26/2017,0:23:57,72.0,15.55
...,...,...,...,...
2017-08-09 07:45:05,8/9/2017,7:45:05,160.0,31.81
2017-08-09 07:50:05,8/9/2017,7:50:05,157.0,31.56
2017-08-09 07:55:05,8/9/2017,7:55:05,158.0,32.12


In [58]:
list2 = (
    manual_mode_data_collected_dataframe_index.groupby('Date')['Sensor Glucose (mg/dL)']
    .count()
    .where(lambda x:x>0.8*288)
    .dropna()
    .index
    .tolist()
)
list2

['7/26/2017', '8/1/2017', '8/2/2017', '8/3/2017']

In [59]:
manual_mode_data_collected_dataframe_index = (
    manual_mode_data_collected_dataframe_index.loc[manual_mode_data_collected_dataframe_index['Date'].isin(list2)]
)
manual_mode_data_collected_dataframe_index

,Date,Time,Sensor Glucose (mg/dL),ISIG Value
date_time_stamp,,,,
2017-07-26 00:03:57,7/26/2017,0:03:57,65.0,14.45
2017-07-26 00:08:57,7/26/2017,0:08:57,64.0,14.26
2017-07-26 00:13:57,7/26/2017,0:13:57,64.0,14.47
2017-07-26 00:18:57,7/26/2017,0:18:57,67.0,15.06
2017-07-26 00:23:57,7/26/2017,0:23:57,72.0,15.55
...,...,...,...,...
2017-08-03 23:37:32,8/3/2017,23:37:32,130.0,23.28
2017-08-03 23:42:32,8/3/2017,23:42:32,126.0,22.69
2017-08-03 23:47:32,8/3/2017,23:47:32,125.0,22.32


Percentage time in hyperglycemia (CGM > 180 mg/dL) - wholeday, daytime, and overnight
- For data collected in Manual Mode

In [60]:
percentage_time_in_hyperglycemia_during_wholeday_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] > 180]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_during_wholeday_when_data_collected_manually

Date
7/26/2017    37.152778
8/1/2017     21.875000
8/2/2017     17.013889
8/3/2017     48.611111
Name: Sensor Glucose (mg/dL), dtype: float64

In [61]:
percentage_time_in_hyperglycemia_during_daytime_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] > 180]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_during_daytime_when_data_collected_manually

Date
7/26/2017    37.152778
8/1/2017     17.708333
8/2/2017     17.013889
8/3/2017     29.513889
Name: Sensor Glucose (mg/dL), dtype: float64

In [62]:
percentage_time_in_hyperglycemia_during_overnight_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] > 180]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_during_overnight_when_data_collected_manually

Date
8/1/2017     4.166667
8/3/2017    19.097222
Name: Sensor Glucose (mg/dL), dtype: float64

percentage of time in hyperglycemia critical (CGM > 250 mg/dL) - wholeday, daytime, and overnight

- For data collected in Manual Mode

In [63]:
percentage_time_in_hyperglycemia_critical_during_wholeday_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] > 250]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_critical_during_wholeday_when_data_collected_manually

Date
7/26/2017    16.666667
8/3/2017     27.777778
Name: Sensor Glucose (mg/dL), dtype: float64

In [64]:
percentage_time_in_hyperglycemia_critical_during_daytime_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] > 250]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_critical_during_daytime_when_data_collected_manually

Date
7/26/2017    16.666667
8/3/2017     17.708333
Name: Sensor Glucose (mg/dL), dtype: float64

In [65]:
percentage_time_in_hyperglycemia_critical_during_overnight_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] > 250]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hyperglycemia_critical_during_overnight_when_data_collected_manually

Date
8/3/2017    10.069444
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL) - wholeday, daytime, and overnight

- For data collected in Manual Mode

In [66]:
percentage_time_in_range_during_wholeday_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] >= 70) & (manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] <= 180)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_during_wholeday_when_data_collected_manually

Date
7/26/2017    51.736111
8/1/2017     78.125000
8/2/2017     79.166667
8/3/2017     45.833333
Name: Sensor Glucose (mg/dL), dtype: float64

In [67]:
percentage_time_in_range_during_daytime_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] >= 70) & (manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] <= 180)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_during_daytime_when_data_collected_manually

Date
7/26/2017    28.125000
8/1/2017     57.291667
8/2/2017     54.166667
8/3/2017     39.930556
Name: Sensor Glucose (mg/dL), dtype: float64

In [68]:
percentage_time_in_range_during_overnight_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] >= 70) & (manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] <= 180)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_during_overnight_when_data_collected_manually

Date
7/26/2017    23.611111
8/1/2017     20.833333
8/2/2017     25.000000
8/3/2017      5.902778
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL) - wholeday, daytime, and overnight

- For data collected in Manual Mode

In [69]:
percentage_time_in_range_secondary_during_wholeday_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] >= 70) & (manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] <= 150)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_secondary_during_wholeday_when_data_collected_manually

Date
7/26/2017    44.444444
8/1/2017     47.222222
8/2/2017     63.888889
8/3/2017     34.722222
Name: Sensor Glucose (mg/dL), dtype: float64

In [70]:
percentage_time_in_range_secondary_during_daytime_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] >= 70) & (manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] <= 150)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_secondary_during_daytime_when_data_collected_manually

Date
7/26/2017    24.652778
8/1/2017     38.194444
8/2/2017     41.666667
8/3/2017     34.722222
Name: Sensor Glucose (mg/dL), dtype: float64

In [71]:
percentage_time_in_range_secondary_during_overnight_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[(manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] >= 70) & (manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] <= 150)]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_range_secondary_during_overnight_when_data_collected_manually

Date
7/26/2017    19.791667
8/1/2017      9.027778
8/2/2017     22.222222
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in hypoglycemia level 1 (CGM < 70 mg/dL) - wholeday, daytime, and overnight

- For data collected in Manual Mode

In [72]:
percentage_time_in_hypoglycemia_level1_during_wholeday_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] < 70]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level1_during_wholeday_when_data_collected_manually

Date
7/26/2017    11.111111
8/2/2017      3.819444
8/3/2017      5.555556
Name: Sensor Glucose (mg/dL), dtype: float64

In [73]:
percentage_time_in_hypoglycemia_level1_during_daytime_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] < 70]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level1_during_daytime_when_data_collected_manually

Date
7/26/2017    9.722222
8/2/2017     3.819444
8/3/2017     5.555556
Name: Sensor Glucose (mg/dL), dtype: float64

In [74]:
percentage_time_in_hypoglycemia_level1_during_overnight_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] < 70]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level1_during_overnight_when_data_collected_manually

Date
7/26/2017    1.388889
Name: Sensor Glucose (mg/dL), dtype: float64

percentage time in hypoglycemia level 2 (CGM < 54 mg/dL) - wholeday, daytime, and overnight

- For data collected in Manual Mode

In [75]:
percentage_time_in_hypoglycemia_level2_during_wholeday_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] < 54]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level2_during_wholeday_when_data_collected_manually

Date
7/26/2017    6.250000
8/3/2017     2.430556
Name: Sensor Glucose (mg/dL), dtype: float64

In [76]:
percentage_time_in_hypoglycemia_level2_during_daytime_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('6:00:00','23:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] < 54]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level2_during_daytime_when_data_collected_manually

Date
7/26/2017    6.250000
8/3/2017     2.430556
Name: Sensor Glucose (mg/dL), dtype: float64

In [77]:
percentage_time_in_hypoglycemia_level2_during_overnight_when_data_collected_manually = (
    manual_mode_data_collected_dataframe_index.between_time('0:00:00','05:59:59')[['Date','Time','Sensor Glucose (mg/dL)']]
    .loc[manual_mode_data_collected_dataframe_index['Sensor Glucose (mg/dL)'] < 54]
    .groupby('Date')['Sensor Glucose (mg/dL)']
    .count()/288*100
)
percentage_time_in_hypoglycemia_level2_during_overnight_when_data_collected_manually

Series([], Name: Sensor Glucose (mg/dL), dtype: float64)

convert to a dataframe with all values in auto mode and manual mode

In [78]:
results_data_frame = pd.DataFrame({
    
    'percentage_time_in_hyperglycemia_during_overnight':[ 
        percentage_time_in_hyperglycemia_during_overnight_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hyperglycemia_during_overnight_when_data_collected_in_auto_mode.mean(axis=0)
    ],

    'percentage_time_in_hyperglycemia_critical_during_overnight':[ 
        percentage_time_in_hyperglycemia_critical_during_overnight_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hyperglycemia_critical_during_overnight_when_data_collected_in_auto_mode.mean(axis=0)
    ],


    'percentage_time_in_range_during_overnight':[ 
        percentage_time_in_range_during_overnight_when_data_collected_manually.mean(axis=0),
        percentage_time_in_range_during_overnight_when_data_collected_in_auto_mode.mean(axis=0)
    ],


    'percentage_time_in_range_secondary_during_overnight':[ 
        percentage_time_in_range_secondary_during_overnight_when_data_collected_manually.mean(axis=0),
        percentage_time_in_range_secondary_during_overnight_when_data_collected_in_auto_mode.mean(axis=0)
    ],


    'percentage_time_in_hypoglycemia_level1_during_overnight':[ 
        percentage_time_in_hypoglycemia_level1_during_overnight_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hypoglycemia_level1_during_overnight_when_data_collected_in_auto_mode.mean(axis=0)
    ],


    'percentage_time_in_hypoglycemia_level2_during_overnight':[ 
        np.nan_to_num(percentage_time_in_hypoglycemia_level2_during_overnight_when_data_collected_manually.mean(axis=0)),
        percentage_time_in_hypoglycemia_level2_during_overnight_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hyperglycemia_during_daytime':[ 
        percentage_time_in_hyperglycemia_during_daytime_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hyperglycemia_during_daytime_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hyperglycemia_critical_during_daytime':[ 
        percentage_time_in_hyperglycemia_critical_during_daytime_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hyperglycemia_critical_during_daytime_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_range_during_daytime':[ 
        percentage_time_in_range_during_daytime_when_data_collected_manually.mean(axis=0),
        percentage_time_in_range_during_daytime_when_data_collected_in_auto_mode.mean(axis=0)
    ],
    
                           
    'percentage_time_in_range_secondary_during_daytime':[ 
        percentage_time_in_range_secondary_during_daytime_when_data_collected_manually.mean(axis=0),
        percentage_time_in_range_secondary_during_daytime_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hypoglycemia_level1_during_daytime':[ 
        percentage_time_in_hypoglycemia_level1_during_daytime_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hypoglycemia_level1_during_daytime_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hypoglycemia_level2_during_daytime':[ 
        percentage_time_in_hypoglycemia_level2_during_daytime_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hypoglycemia_level2_during_daytime_when_data_collected_in_auto_mode.mean(axis=0)
    ],

                           
                           
    'percentage_time_in_hyperglycemia_during_wholeday':[ 
        percentage_time_in_hyperglycemia_during_wholeday_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hyperglycemia_during_wholeday_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hyperglycemia_critical_during_wholeday':[ 
        percentage_time_in_hyperglycemia_critical_during_wholeday_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hyperglycemia_critical_during_wholeday_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                          
    'percentage_time_in_range_during_wholeday':[ 
        percentage_time_in_range_during_wholeday_when_data_collected_manually.mean(axis=0),
        percentage_time_in_range_during_wholeday_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_range_secondary_during_wholeday':[ 
        percentage_time_in_range_secondary_during_wholeday_when_data_collected_manually.mean(axis=0),
        percentage_time_in_range_secondary_during_wholeday_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hypoglycemia_level1_during_wholeday':[ 
        percentage_time_in_hypoglycemia_level1_during_wholeday_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hypoglycemia_level1_during_wholeday_when_data_collected_in_auto_mode.mean(axis=0)
    ],
                           
    'percentage_time_in_hypoglycemia_level2_during_wholeday':[ 
        percentage_time_in_hypoglycemia_level2_during_wholeday_when_data_collected_manually.mean(axis=0),
        percentage_time_in_hypoglycemia_level2_during_wholeday_when_data_collected_in_auto_mode.mean(axis=0)
    ]
                    
                          
},
                          index=['manual_mode','auto_mode'])

In [79]:
results_data_frame

,percentage_time_in_hyperglycemia_during_overnight,percentage_time_in_hyperglycemia_critical_during_overnight,percentage_time_in_range_during_overnight,percentage_time_in_range_secondary_during_overnight,percentage_time_in_hypoglycemia_level1_during_overnight,percentage_time_in_hypoglycemia_level2_during_overnight,percentage_time_in_hyperglycemia_during_daytime,percentage_time_in_hyperglycemia_critical_during_daytime,percentage_time_in_range_during_daytime,percentage_time_in_range_secondary_during_daytime,percentage_time_in_hypoglycemia_level1_during_daytime,percentage_time_in_hypoglycemia_level2_during_daytime,percentage_time_in_hyperglycemia_during_wholeday,percentage_time_in_hyperglycemia_critical_during_wholeday,percentage_time_in_range_during_wholeday,percentage_time_in_range_secondary_during_wholeday,percentage_time_in_hypoglycemia_level1_during_wholeday,percentage_time_in_hypoglycemia_level2_during_wholeday
manual_mode,11.631944,10.069444,18.836806,17.013889,1.388889,0.0,25.347222,17.187500,44.878472,34.809028,6.365741,4.340278,31.163194,22.222222,63.715278,47.569444,6.828704,4.340278
auto_mode,6.543803,4.745370,21.807796,19.027778,2.314815,NaN,19.802867,7.326389,51.635305,38.328853,5.257937,1.629274,22.547043,8.038194,73.443100,56.742832,5.404589,1.629274


In [80]:
results_data_frame.to_csv('Results.csv',header=False,index=False)